In [1]:
import os
import sys
import pandas as pd
import numpy as np
from sklearn.neighbors import DistanceMetric
import math
from math import sin, cos, sqrt, atan2, radians

In [2]:
head, tail = os.path.split(os.getcwd())
head, tail

('/Users/Gagandeep/Desktop/Concordia/12 Winter 2021/SOEN 471/NoToW',
 'notebooks')

In [3]:
head, tail = os.path.split(os.getcwd())
data_dir = os.path.join(head, 'data')
data_raw_dir = os.path.join(data_dir, 'raw')
data_raw_dir

'/Users/Gagandeep/Desktop/Concordia/12 Winter 2021/SOEN 471/NoToW/data/raw'

In [4]:
os.listdir(data_raw_dir)

['en_climate_daily_QC_702S006_2017_P1D.csv',
 'Periodes.csv',
 'remorquages.csv',
 'en_climate_daily_QC_702S006_2020_P1D.csv',
 'en_climate_daily_QC_702S006_2016_P1D.csv',
 'montreal_boroughs.geojson',
 'headers.txt',
 '.gitkeep',
 'en_climate_daily_QC_702S006_2015_P1D.csv',
 'BornesHorsRue.csv',
 'BornesSurRue.csv',
 'ReglementationPeriode.csv',
 'Places.csv',
 'boroughs.txt.npy',
 'en_climate_daily_QC_702S006_2019_P1D.csv',
 'Reglementations.csv',
 'remorquages2005-2015.csv',
 'en_climate_daily_QC_702S006_2018_P1D.csv',
 'EmplacementReglementation.csv']

In [5]:
df_remor = pd.read_csv(os.path.join(data_raw_dir, 'remorquages.csv'), header=0)
df_remor.head()

,DATE_ORIGINE,LONGITUDE_ORIGINE,LATITUDE_ORIGINE,RUE_ORIGINE,SECTEUR_ORIGINE,ARRONDISSEMENT_ORIGINE,DATE_DESTINATION,LONGITUDE_DESTINATION,LATITUDE_DESTINATION,RUE_DESTINATION,SECTEUR_DESTINATION,ARRONDISSEMENT_DESTINATION,MOTIF_REMORQUAGE
0,2015-12-31T00:23:00,-73.587203,45.524981,rue Drolet,NaN,Plateau-Mont-Royal,2015-12-31T00:25:00,-73.586791,45.525131,rue de Grand-Pré,NaN,Plateau-Mont-Royal,Constat d'infraction
1,2015-12-31T00:24:00,-73.622256,45.585753,boulevard des Grandes-Prairies,NaN,Saint-Léonard,2015-12-31T00:30:00,-73.621005,45.586975,boulevard des Grandes-Prairies,NaN,Saint-Léonard,Constat d'infraction
2,2015-12-31T00:25:00,-73.600692,45.535027,rue Saint-Hubert,NaN,Rosemont - La Petite-Patrie,2015-12-31T00:32:00,-73.600881,45.535163,rue Saint-Hubert,NaN,Rosemont - La Petite-Patrie,Constat d'infraction
3,2015-12-31T00:25:00,-73.620824,45.516675,avenue Ducharme,NaN,Outremont,2015-12-31T00:28:00,-73.621995,45.518764,avenue Rockland,NaN,Outremont,Constat d'infraction
4,2015-12-31T00:27:00,-73.580197,45.494309,rue Saint-Mathieu,NaN,Ville-Marie,2015-12-31T00:39:00,-73.579972,45.493133,rue Sainte-Catherine Ouest,NaN,Ville-Marie,Constat d'infraction


In [6]:
list(df_remor.dtypes)

[dtype('O'),
 dtype('float64'),
 dtype('float64'),
 dtype('O'),
 dtype('float64'),
 dtype('O'),
 dtype('O'),
 dtype('float64'),
 dtype('float64'),
 dtype('O'),
 dtype('float64'),
 dtype('O'),
 dtype('O')]

In [7]:
df_remor.isnull().any()

DATE_ORIGINE                  False
LONGITUDE_ORIGINE              True
LATITUDE_ORIGINE               True
RUE_ORIGINE                   False
SECTEUR_ORIGINE                True
ARRONDISSEMENT_ORIGINE         True
DATE_DESTINATION               True
LONGITUDE_DESTINATION          True
LATITUDE_DESTINATION           True
RUE_DESTINATION               False
SECTEUR_DESTINATION            True
ARRONDISSEMENT_DESTINATION     True
MOTIF_REMORQUAGE               True
dtype: bool

In [8]:
df_remor['MOTIF_REMORQUAGE'].value_counts()

Constat d'infraction        240333
Remis au propriétaire         4576
Autre                         2485
Déplacement non autorisé      1746
Événement policier             774
Name: MOTIF_REMORQUAGE, dtype: int64

## Unique boroughs

In [27]:
boroughs = df_remor['ARRONDISSEMENT_ORIGINE']
print(type(boroughs))
print(boroughs.isnull().sum())
remove_nan = boroughs.dropna()
print(type(remove_nan))
print(remove_nan.isnull().sum())
print(remove_nan.unique())
# remove_nan.unique().save(os.path.join(data_raw_dir, 'boroughs.txt'))

np.save(os.path.join(data_raw_dir, 'boroughs.txt'), remove_nan.unique())

<class 'pandas.core.series.Series'>
741
<class 'pandas.core.series.Series'>
0
['Plateau-Mont-Royal' 'Saint-Léonard' 'Rosemont - La Petite-Patrie'
 'Outremont' 'Ville-Marie' 'Mercier - Hochelaga-Maisonneuve' 'LaSalle'
 'Villeray - Saint-Michel - Parc-Extension'
 'Rivière-des-Prairies - Pointe-aux-Trembles'
 'Côte-des-Neiges - Notre-Dame-de-Grâce' 'Lachine' 'Saint-Laurent'
 'Ahuntsic - Cartierville' 'Sud-Ouest' 'Anjou' 'Montréal-Nord' 'Verdun'
 'Pierrefonds - Roxboro']


## Probability of a spot

In [9]:
df_prob = df_remor[['LONGITUDE_ORIGINE', 'LATITUDE_ORIGINE', 'LONGITUDE_DESTINATION', 'LATITUDE_DESTINATION']]

In [10]:
df_prob = df_prob.round(5)

In [11]:
df_prob

,LONGITUDE_ORIGINE,LATITUDE_ORIGINE,LONGITUDE_DESTINATION,LATITUDE_DESTINATION
0,-73.58720,45.52498,-73.58679,45.52513
1,-73.62226,45.58575,-73.62100,45.58697
2,-73.60069,45.53503,-73.60088,45.53516
3,-73.62082,45.51668,-73.62200,45.51876
4,-73.58020,45.49431,-73.57997,45.49313
...,...,...,...,...
250072,-73.70518,45.53120,-73.70428,45.53077
250073,-73.65453,45.55993,-73.65046,45.56053
250074,-73.57954,45.49135,-73.57942,45.49364
250075,-73.59578,45.46153,-73.59017,45.46118


In [12]:
df_prob['LONGITUDE_ORIGINE'] = df_prob['LONGITUDE_ORIGINE'].astype(str)
df_prob['LATITUDE_ORIGINE'] = df_prob['LATITUDE_ORIGINE'].astype(str)
df_prob['LONGITUDE_DESTINATION'] = df_prob['LONGITUDE_DESTINATION'].astype(str)
df_prob['LATITUDE_DESTINATION'] = df_prob['LATITUDE_DESTINATION'].astype(str)

In [13]:
df_prob['ORIGIN'] =  df_prob['LATITUDE_ORIGINE'] +','+ df_prob['LONGITUDE_ORIGINE']
df_prob['DEST'] = df_prob['LATITUDE_DESTINATION'] +','+ df_prob['LONGITUDE_DESTINATION']
df_prob['COUNT'] = 1
# df_prob = df_prob[['DATE_ORIGINE', 'ORIGIN', 'DEST', 'COUNT']]

In [14]:
# df_prob['ORIGIN'].count()
# print()

In [15]:
df_prob_count = df_prob[['ORIGIN', 'COUNT']]
df_prob_count.groupby(['ORIGIN']).count().reset_index()
df_prob_count

,ORIGIN,COUNT
0,"45.52498,-73.5872",1
1,"45.58575,-73.62226",1
2,"45.53503,-73.60069",1
3,"45.51668,-73.62082",1
4,"45.49431,-73.5802",1
...,...,...
250072,"45.5312,-73.70518",1
250073,"45.55993,-73.65453",1
250074,"45.49135,-73.57954",1
250075,"45.46153,-73.59578",1


In [16]:
g = df_prob_count.groupby(['ORIGIN']).sum().reset_index()

In [17]:
g[['ORIGIN', 'COUNT']]

,ORIGIN,COUNT
0,"45.41524,-73.6236",1
1,"45.41528,-73.62384",1
2,"45.41529,-73.62144",1
3,"45.41529,-73.62369",1
4,"45.41531,-73.62383",1
...,...,...
242725,"45.70096,-73.48169",1
242726,"45.70121,-73.48097",1
242727,"45.70135,-73.48046",1
242728,"45.70145,-73.48005",1


## Distance between pick up and drop off using Haversin formula
```https://www.kite.com/python/answers/how-to-find-the-distance-between-two-lat-long-coordinates-in-python```

In [6]:
df_avg_drop_dist = df_remor[['LONGITUDE_ORIGINE', 'LATITUDE_ORIGINE', 'LONGITUDE_DESTINATION', 'LATITUDE_DESTINATION']]
df_avg_drop_dist

,LONGITUDE_ORIGINE,LATITUDE_ORIGINE,LONGITUDE_DESTINATION,LATITUDE_DESTINATION
0,-73.587203,45.524981,-73.586791,45.525131
1,-73.622256,45.585753,-73.621005,45.586975
2,-73.600692,45.535027,-73.600881,45.535163
3,-73.620824,45.516675,-73.621995,45.518764
4,-73.580197,45.494309,-73.579972,45.493133
...,...,...,...,...
250072,-73.705183,45.531197,-73.704284,45.530772
250073,-73.654533,45.559930,-73.650458,45.560528
250074,-73.579544,45.491353,-73.579424,45.493636
250075,-73.595783,45.461529,-73.590168,45.461180


In [ ]:
R = 6373.0

lat1 = np.radians(df_avg_drop_dist['LATITUDE_ORIGINE'])
lon1 = np.radians(df_avg_drop_dist['LONGITUDE_ORIGINE'])
lat2 = np.radians(df_avg_drop_dist['LATITUDE_DESTINATION'])
lon2 = np.radians(df_avg_drop_dist['LONGITUDE_DESTINATION'])

dlon = lon2 - lon1
dlat = lat2 - lat1
# print(dlon, dlat)
a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
# print(type(a))
c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
# print(c)
distance = R * c
# print(type(distance))
df_avg_drop_dist.loc[:, 'avg_dist'] = distance

In [ ]:
df_avg_drop_dist

In [ ]:
df_avg_drop_dist['avg_dist'].mean()